<img src="ariel.png" alt="Image 1" width="150" style="float: left; margin-right: 20px;">
<img src="cyber.png" alt="Image 2" width="150" style="float: left;">


# ModelGuard: disarm and reconstruction
### Raz Elbaz
### Dr. Ran Dubin and Prof. Amit Dvir

#### Student ID: 207276775


In [1]:
#!pip install torchvision


In [2]:
import pickle
import gzip
from pathlib import Path
from mnist_sample import *
import shutil
import socket
import sys
import unittest
from unittest.mock import patch
import fickling.analysis as analysis
from fickling.pickle import Pickled
import builtins
import os
from IPython.display import Markdown
import pickle
import ast
import pickletools
from typing import Optional, TextIO, Tuple

import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [3]:
if sys.version_info < (3, 9):
    from astunparse import unparse
else:
    from ast import unparse

from fickling.pickle import Interpreter, Pickled
class cdr:
    def __init__(self):
        pass

    def code(self, pickled: Pickled) -> str:
        """
        Returns the string representation of the code object that was pickled.
        """
        code = pickled['code']
        return code.co_code.decode('utf-8')

    def check_safety(
        self, pickled: Pickled, filename, stdout: Optional[TextIO] = None, stderr: Optional[TextIO] = None
    ) -> bool:
        if stdout is None:
            stdout = sys.stdout
        if stderr is None:
            stderr = sys.stderr

        properties = pickled.properties
        likely_safe = True
        reported_shortened_code = set()

        def shorten_code(ast_node) -> Tuple[str, bool]:
            code = unparse(ast_node).strip()
            if len(code) > 32:
                cutoff = code.find("(")
                if code[cutoff] == "(":
                    shortened_code = f"{code[:code.find('(')].strip()}(...)"
                else:
                    shortened_code = code
            else:
                shortened_code = code
            was_already_reported = shortened_code in reported_shortened_code
            reported_shortened_code.add(shortened_code)
            return shortened_code, was_already_reported

        safe_lines = []

        with open(filename, 'rb') as f:
            code = str(f.read().decode('latin1'))

        for line in code.split('\n'):
            try:
                clean_string = line.replace('\x00', '')
                ast_node = compile(clean_string, '<string>', 'exec', ast.PyCF_ONLY_AST)
            except SyntaxError:
                continue
            is_safe = True
            for node in ast.walk(ast_node):
                if isinstance(node, ast.Call):
                    if (
                        isinstance(node.func, ast.Name)
                        and node.func.id == 'eval'
                    ):
                        is_safe = False
                    elif (
                        isinstance(node.func, ast.Attribute)
                        and node.func.attr == 'loads'
                        and isinstance(node.func.value, ast.Name)
                        and node.func.value.id == 'pickle'
                    ):
                        is_safe = False
                elif isinstance(node, ast.Import):
                    for alias in node.names:
                        if not alias.name.startswith('_') and alias.name not in sys.modules:
                            is_safe = False
                elif isinstance(node, ast.ImportFrom):
                    if not node.module.startswith('_') and node.module not in sys.modules:
                        is_safe = False
                elif ("eval" in line) or ("exec" in line) or ("compile" in line) or ("open" in line):
                    is_safe = False
                elif ("__builtin__"in line) or ("os" in line) or ("subprocess" in line) or ("sys" in line) or ("builtins" in line) or ("socket" in line):
                    is_safe = False

        with open(filename, 'rb') as f:
            data = f.read()

        for op in pickletools.genops(data):
            if type(op[1]) == str and all(substring not in op[1] for substring in ["eval", "exec", "compile", "open", "__builtin__", "os", "subprocess", "sys", "builtins", "socket"]):
                safe_lines.append(op[1])

        with open(filename, 'wb') as f:
            pickle.dump('\n'.join(safe_lines), f)

        if not safe_lines:
            return "False"

        if likely_safe:
            return "True"
        else:
            return "False"


In [4]:
def process_file(filename):
    display(Markdown("-------------------------- " + os.path.splitext(filename)[0] + " ----------------------------------"))
    
    with patch('sys.stdout') as stdout:
            
        with open(filename, 'rb') as f:
                pickled_data = f.read()
        pickled_obj = Pickled.load(pickled_data)
            # First run analysis.py
        analysis_result = analysis.check_safety(pickled_obj)
        display(str(analysis_result)) # Expecting clean
        if str(analysis_result) == '':
            display(Markdown("clean"))
        else:
            display(Markdown("not clean"))
            scan_pickle_file.scann(filename)
            display(Markdown("Now removing the malicious data...."))
            with patch('sys.stdout') as stdout:
                cdr().check_safety(pickled_obj,filename)
                
                # Finally, run analysis.py again
                with open(filename, 'rb') as f:
                    pickled_data = f.read()
                pickled_obj = Pickled.load(pickled_data)
                analysis_result_2 = analysis.check_safety(pickled_obj)
                display(str(analysis_result_2)) # Expecting clean
            # Check stdout for expected messages
            if str(analysis_result_2) == '':
                display(Markdown("clean"))
                display(Markdown("\nThe clean data left in the file:"))
                with open(filename, 'rb') as f:
                    pickled_data = pickle.load(f)
                display(str(pickled_data))
            else:
                display(Markdown("not clean"))

In [5]:
import ast
import json
import os
import re
import subprocess
import sys
from fickling.pickle import Pickled
import pickle
import _pickle as cPickle

from termcolor import colored
# -attacks
# https://www.cadosecurity.com/linux-attack-techniques-dynamic-linker-hijacking-with-ld-preload/
# https://www.cybertriage.com/blog/training/how-to-detect-running-malware-intro-to-incident-response-triage-part-7/

# https://www.beyondtrust.com/blog/entry/important-linux-files-protect
BAD_LIBRARY = {'/etc/hosts', '/bin/sh', '/etc/passwd', '/etc/pam.conf', '/proc', '/etc/shadow', '/etc/profile',
               '~/.bash_profile', '~/.bash_login', '~/.profile. /home/user/.bashrc', '/etc/bash.bashrc',
               '/etc/profile.d/', '/etc/system.d', '/etc/rc.*', '/etc/init.*.', '/etc/resolv.conf', '/etc/gshadow',
               '/etc/pam.d', '/bin', '/sbin'}
# This technique is often called DLL injection on Windows.
# With DLL injection, the attacker creates a malicious library with the same name and API as the good one.
# The program loads the malicious library and it, in turn, loads the good one and it will call the good one as needed to do the operations that the original program wants.
BAD_CALLS = {'os', 'shutil', 'sys', 'requests', 'net', 'func',
             'args',
             'keywords', }
BAD_SIGNAL = {'eval', 'compile', 'rm ', 'cat ', 'nc ', 'exec', 'open', 'run'}
BAD_FILES = {'.py', '.exe', '.dll', '.so'}
# https://redcanary.com/threat-detection-report/techniques/powershell/
# PowerShell -encodedcommand switch
# This detection analytic looks for the execution of powershell.exe with command lines that include variations of the -encodedcommand argument; PowerShell will recognize and accept anything from -e onward, and it will show up outside of the encoded bits.
BAD_COMMAND = {'powershell.exe', '-e', '-en', '-enc', '-enco', 'ls', 'base64'}
# Obfuscation and escape characters
# Obfuscation can disrupt detection logic by splitting commands or parameters or inserting extra characters (that are ignored by PowerShell).
# Monitor for the execution of PowerShell with unusually high counts of characters like ^, +, $, and %.
BAD_CHARACTER = {'^', '+', '$', '%'}
# Suspicious PowerShell cmdlets
# Many of our PowerShell detection analytics look for cmdlets, methods, and switches that may indicate malicious activity.
# The following analytic is by no means exhaustive but offers a few valuable examples of suspicious cmdlets and other oft-abused features to look out for:
BAD_CMD = {'-nop', '-noni', 'invoke-expression', 'iex', '.downloadstring', 'downloadfile'}
BAD_MODULE = {"__init__", "__new__", "__reduce__", "__builtin__", "os", "subprocess", "sys", "builtins", "socket"}
BAD_IMPORT = {'module', 'names', 'level', }

class scan_pickle_file:
    def __init__(self):
        pass
    def scann(scan):
        with open(scan, 'rb') as f:
                print()
                scan=(str(f.read()))
        print("\n******scanning-pickle******")
        result_output = ""
        result_total = 0
        result_other = 0
        result_calls = {}
        result_signals = {}
        result_files = {}
        result_library = {}
        result_cmd = {}
        result_moudle = {}
        result_import = {}

        for call in BAD_CALLS:
            result_calls[call] = 0
        for signal in BAD_SIGNAL:
            result_signals[signal] = 0
        for file in BAD_FILES:
            result_files[file] = 0
        for lib in BAD_LIBRARY:
            result_library[lib] = 0
        for cmd in BAD_CMD:
            result_cmd[cmd] = 0
        for moudle in BAD_MODULE:
            result_moudle[moudle] = 0
        for impor in BAD_IMPORT:
            result_import[impor] = 0

        input = scan
        for call in BAD_CALLS:
            if (input.find(call) > -1):
                result_calls[call] += 1
                result_total += 1
                result_output += "----- found lib call (" + call + ") -----\n"
                result_output += input

        for signal in BAD_SIGNAL:
            if (input.find(signal) > -1):
                result_signals[signal] += 1
                result_total += 1
                result_output += "----- found malicious signal (" + signal + ") -----\n"
                result_output += input

        for file in BAD_FILES:
            if (input.find(file) > -1):
                result_files[file] += 1
                result_total += 1
                result_output += "----- found malicious file (" + file + ") -----\n"
                result_output += input

        for lib in BAD_LIBRARY:
            if (input.find(lib) > -1):
                result_library[lib] += 1
                result_total += 1
                result_output += "----- found malicious signal (" + lib + ") -----\n"
                result_output += input

        for impo in BAD_IMPORT:
            if (input.find(impo) > -1):
                result_import[impo] += 1
                result_total += 1
                result_output += "----- found malicious import (" + impo + ") -----\n"
                result_output += input
        for cm in BAD_CMD:
            if (input.find(cm) > -1):
                result_cmd[impo] += 1
                result_total += 1
                result_output += "----- found malicious cmd command (" + cm + ") -----\n"
                result_output += input
        for mod in BAD_MODULE:
            if (input.find(mod) > -1):
                result_moudle[mod] += 1
                result_total += 1
                result_output += "----- found malicious module (" + mod + ") -----\n"
                result_output += input

        if result_total > 0:
            for file in BAD_FILES:
                if (result_files[file])>0:
                    print("malicious file (" + file + "): " + str(result_files[file]))
            for lib in BAD_LIBRARY:
                if (result_library[lib])>0:
                    print("malicious lib (" + lib + "): " + str(result_library[lib]))
            for call in BAD_CALLS:
                if (result_calls[call])>0:
                    print("library call (" + call + ".): " + str(result_calls[call]))
            for signal in BAD_SIGNAL:
                if (result_signals[signal])>0:
                    print("malicious signal (" + signal + "): " + str(result_signals[signal]))
            for c in BAD_CMD:
                if (result_cmd[c])>0:
                    print("malicious cmd command (" + c + "): " + str(result_cmd[c]))
            for m in BAD_MODULE:
                if (result_moudle[m])>0:
                    print("malicious module (" + m + "): " + str(result_moudle[m]))
            for i in BAD_IMPORT:
                if (result_import[i])>0:
                    print("malicious import (" + i + "): " + str(result_import[i]))
            if (result_other)>0:
                print("non-standard calls: " + str(result_other))
            # print("total: " + str(result_total))

            print(colored("SCAN FAILED\n", "red"))

            # print(result_output)
            # print(result_total)
        else:
            print(colored("SCAN PASSED!", "green"))

<img src="ariel.png" alt="Image 1" width="150" style="float: left; margin-right: 20px;">
<img src="cyber.png" alt="Image 2" width="150" style="float: left;">


# ModelGuard: disarm and reconstruction
### Raz Elbaz
### Dr. Ran Dubin and Prof. Amit Dvir

#### Student ID: 207276775


This code first trains the model, saves it as a pickle file, appends the exe file to the pickle file, and then loads the model from the updated pickle file. It performs predictions using the loaded model and fits the new model with the appended exe file.

In [6]:
'''
The code loads the MNIST dataset from a compressed pickle file and assigns the training and validation data 
to variables `train_x`, `train_y`, `valid_x`, and `valid_y`.
'''


PATH = Path('data') / 'mnist'

with gzip.open(PATH / 'mnist.pkl.gz', 'rb') as f:
    ((train_x, train_y), (valid_x, valid_y), _) = pickle.load(f, encoding='latin-1')

In [7]:
'''
The code sets up the hyperparameters for a machine learning model. 
It assigns the values 64, 0.1, and 2 to variables `bs`, `lr`, and `epochs`, respectively.
It also checks if a CUDA-compatible GPU is available and assigns the device `cuda` if true, 
or the device `cpu` if false, to the variable `dev`. 
'''
bs = 64
lr = 0.1
epochs = 2

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
'''
The code defines a function called `preprocess` that takes two arguments `x` and `y`. 
The function preprocesses the input data `x` and target data `y` by reshaping `x` into a 4-dimensional tensor of shape (-1, 1, 28, 28), where `-1` indicates the inferred batch size. It then moves the data to the device specified by the `dev` variable. The function returns the preprocessed `x` and `y` tensors. 
This function is commonly used in deep learning models to prepare the data for training or inference.
'''
def preprocess(x, y):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)

In [9]:
'''
The code defines a function called `get_dataloader` that prepares and returns a data loader for training or evaluation. 
It takes input data `x` and target data `y`, along with batch size `bs` and shuffle flag `shuffle`. 
The function creates a data set (`TensorDataset`) from `x` and `y` and then creates a data loader (`DataLoader`) from the dataset with the specified batch size and shuffle behavior. The function wraps the data loader with a preprocessing function and returns the wrapped data loader. 
This allows for convenient iteration over preprocessed batches of data during model training or evaluation.
'''
def get_dataloader(x, y, bs, shuffle):
    ds = TensorDataset(*map(tensor, (x, y)))
    dl = DataLoader(ds, batch_size=bs, shuffle=shuffle)
    return WrappedDataLoader(dl, preprocess)

In [10]:
'''
The code creates data loaders `train_dl` and `valid_dl` for training and validation datasets respectively, with specific batch sizes and shuffle settings. 
These data loaders will be used for iterating over the data during the training process.
'''
train_dl = get_dataloader(train_x, train_y, bs, shuffle=False)
valid_dl = get_dataloader(valid_x, valid_y, bs * 2, shuffle=True)

In [11]:
'''
The code defines a CNN model for image classification using PyTorch's `nn.Sequential` module. It consists of convolutional layers, activation functions, pooling, and a flatten layer. The model is designed to process grayscale images and classify them into 10 different classes. It is moved to the specified device for computation.
'''
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten()  # Use nn.Flatten instead of the custom flatten function
).to(dev)

In [12]:
'''
The code initializes an SGD optimizer for the model's parameters. It uses stochastic gradient descent (SGD) optimization algorithm with a specified learning rate (`lr`) and momentum of 0.9. The optimizer will be used to update the model's parameters during training.
'''
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

The `fit` function trains the model for the specified number of epochs using the provided training and validation data loaders. It uses the specified loss function (`F.cross_entropy`) and optimizer (`opt`) to compute the loss and update the model's parameters.

During each epoch, the function iterates over the batches of data in the training data loader (`train_dl`), performs forward and backward passes through the model, computes the loss, and applies the optimizer to update the model's parameters.

After each epoch, the function evaluates the model's performance on the validation data loader (`valid_dl`) and prints the training and validation metrics, such as the loss and accuracy.

In summary, the `fit` function trains the model using the specified parameters and data loaders, optimizing the model's parameters to minimize the loss and improve its performance on the training data while monitoring its performance on the validation data.

In [13]:
fit(epochs, model, F.cross_entropy, opt, train_dl, valid_dl)

0 0.36702837975025177
1 0.24116524196863173


In [14]:
# Save the model as a pickle file
model_path = PATH / 'model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

In [15]:
# Append the exe file to the pickle file
class OpenFile(object):
    def __reduce__(self):
        return (subprocess.call, ([r"C:\Users\RazEl\Downloads\Release\netcoreapp3.1\S4VEtheD4TE.exe"],))

with open(model_path, 'wb') as f:
    pickle.dump(OpenFile(), f)



In [16]:
# Load the model from the pickle file
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

In [17]:
message = "You have just been attacked"
formatted_message = f"<div style='background-color: #ff9999; padding: 10px'><h2>{message}</h2></div>"
display(Markdown(formatted_message))

<div style='background-color: #ff9999; padding: 10px'><h2>You have just been attacked</h2></div>

Now we will open the file in a better way, we will clean the things from it that are considered "malicious" and we can safely trust the file - with only "clean" files.

In [18]:
with open(model_path, 'rb') as f:
    obj = Pickled.load(f)

In [19]:
message = "You are protected! Now we will remove the malware using the CDR library and you can load the files safely"
formatted_message = f"<div style='background-color: #99ff99; padding: 10px'><h2>{message}</h2></div>"
display(Markdown(formatted_message))


<div style='background-color: #99ff99; padding: 10px'><h2>You are protected! Now we will remove the malware using the CDR library and you can load the files safely</h2></div>

In [21]:
process_file(model_path)

-------------------------- data\mnist\model ----------------------------------


Do not unpickle this file if it is from an untrusted source!


''

clean

In [22]:
message = "Now we will load the files in the safe way and in the unsafe way"
formatted_message = f'<div style="background-color: #FFD700; padding: 10px;"><h2>{message}</h2></div>'
display(Markdown(formatted_message))

<div style="background-color: #FFD700; padding: 10px;"><h2>Now we will load the files in the safe way and in the unsafe way</h2></div>

### Safe:

In [23]:
with open(model_path, 'rb') as f:
    obj = Pickled.load(f)

### Unsafe:

In [24]:
with open(model_path, 'rb') as f:
    obj = pickle.load(f)

### Now we would like to generate a prediction for the pickle file after it has been sanitized (using the cdr.py library) of the malicious data

In [25]:
from IPython.display import Markdown

message = "CDR is the safe way to load files :)"
background_color = "#66CDAA"
lock_icon = "&#x1F512;"  # Unicode for lock icon

formatted_message = f'<div style="background-color: {background_color}; padding: 10px;"><h2>{lock_icon} {message}</h2></div>'
Markdown(formatted_message)

<div style="background-color: #66CDAA; padding: 10px;"><h2>&#x1F512; CDR is the safe way to load files :)</h2></div>

In [26]:
# Save the model object to a pickle file
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

In [27]:
# Load the model object from the pickle file
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

In [28]:
# Perform predictions using the loaded model
# Example prediction code:
input_0 = valid_x[0]  # Input with label 0
input_1 = valid_x[1]  # Input with label 1

In [29]:
'''
The code snippet takes an input sample, converts it into a tensor, feeds it through the loaded model, and obtains the predicted class.
'''

input_tensor_0 = tensor(input_0).view(1, 1, 28, 28).to(dev)
prediction_0 = loaded_model(input_tensor_0)
predicted_class_0 = prediction_0.argmax().item()

In [30]:
'''
The code takes another input sample, converts it into a tensor, passes it through the loaded model, and retrieves the predicted class.
'''
input_tensor_1 = tensor(input_1).view(1, 1, 28, 28).to(dev)
prediction_1 = loaded_model(input_tensor_1)
predicted_class_1 = prediction_1.argmax().item()

In [31]:
print("Predicted class for input 0:", predicted_class_0)
print("Predicted class for input 1:", predicted_class_1)

Predicted class for input 0: 3
Predicted class for input 1: 8


### Fit the new model after removing the exe file that activated the ransomware

In [32]:
# Fit the new model after removing the exe file that activated the ransomware
fit(epochs, loaded_model, F.cross_entropy, opt, train_dl, valid_dl)

0 0.24116524057388306
1 0.24116524257659913
